In [3]:
import sys
sys.path.append("/opt/drake/lib/python3.10/site-packages")  # The jammy dockerfile needs this.

import numpy as np
from pydrake.all import RobotDiagramBuilder, HybridMultibodyCollocation, Solve, eq
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    DirectCollocation,
    FiniteHorizonLinearQuadraticRegulatorOptions,
    FiniteHorizonLinearQuadraticRegulator,
    LogVectorOutput,
    MakeFiniteHorizonLinearQuadraticRegulator,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Parser,
    PiecewisePolynomial,
    PlanarSceneGraphVisualizer,
    SceneGraph,
    Simulator,
    Solve,
    TrajectorySource,
)

from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         Linearize, LinearQuadraticRegulator, LogVectorOutput,
                         MeshcatVisualizer, ModelVisualizer, Parser, Simulator, StartMeshcat,
                         MultibodyPlant, InitializeAutoDiffTuple, ExtractGradient, TrajectorySource,
                         FixedOffsetFrame, QuaternionFloatingJoint, UniformGravityFieldElement,
                         ExternallyAppliedSpatialForce, AbstractValue, ContactModel, GeometrySet,
                         CollisionFilterDeclaration, ContactResults, SpatialAcceleration, SpatialForce,
                         SceneGraph, DiagramBuilder_, AutoDiffXd)

np.set_printoptions(suppress=True, precision=3)

In [4]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()
meshcat.Set2dRenderMode()

Installing NginX server for MeshCat on Deepnote...
INFO:drake:Meshcat listening for connections at https://067a6c9a-b930-4c28-8084-e86f86716a80.deepnoteproject.com/7000/


## URDFs

In [5]:
climber = """
<?xml version="1.0"?><robot name="Climber">
  <link name="world">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 0" />
      </material>
    </visual>
  </link>

  <link name="hand">
    <inertial>
      <origin xyz="0 0 0" />
      <mass value="1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="1 0.647 0 1" />
      </material>
    </visual>

    <collision>
      <geometry>
        <sphere radius=".05" />
      </geometry>
    </collision>
  </link>

  <link name="arm0">
    <inertial>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <mass value="1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <geometry>
        <cylinder radius=".025" length="0.5"/>
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>
  </link>

  <link name="arm1">
    <inertial>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <mass value="1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.25" rpy="0 0 0" />
      <geometry>
        <cylinder radius=".025" length="0.5"/>
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>
  </link>

  <link name="arm2">
    <inertial>
      <origin xyz="0 0 -1" rpy="0 0 0" />
      <mass value="1" />
      <inertia ixx="1e-3 " ixy="0" ixz="0" iyy="1e-3 " iyz="0" izz="1e-3 " />
    </inertial>

    <visual>
      <origin xyz="0 0 -1" rpy="0 0 0" />
      <geometry>
        <sphere radius=".05" />
      </geometry>
      <material>
        <color rgba="0 0 0 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0 0" />
      <geometry>
        <cylinder radius=".025" length="1"/>
      </geometry>
      <material>
        <color rgba="0.5 0.5 0.5 1" />
      </material>
    </visual>
  </link>

  <joint name="world-hand" type="planar">
    <parent link="world" />
    <child link="hand" />
    <axis xyz="0 1 0" />
  </joint>
  
  <joint name="theta0" type="continuous">
    <parent link="hand" />
    <child link="arm0" />
    <axis xyz="0 -1 0" />
  </joint>

  <joint name="theta1" type="revolute">
    <origin xyz="0 0 -0.5" rpy="0 0 0"/>
    <parent link="arm0" />
    <child link="arm1" />
    <axis xyz="0 -1 0" />
    <limit effort="10000.0" lower="-2.7" upper="0.0" velocity="10000.0"/>
  </joint>

  <joint name="theta2" type="continuous">
    <origin xyz="0 0 -0.5" rpy="0 0 0"/>
    <parent link="arm1" />
    <child link="arm2" />
    <axis xyz="0 -1 0" />
  </joint>
  
  <transmission type="SimpleTransmission" name="theta1_force">
    <actuator name="force1" />
    <joint name="theta1" />
  </transmission>

  <transmission type="SimpleTransmission" name="theta2_force">
    <actuator name="force2" />
    <joint name="theta2" />
  </transmission>
  
  </robot>
</xml>
"""

In [6]:
obstacle_background = """
<?xml version="1.0"?><robot name="Obstacle_Background">
  <link name="obstacle_base_background">
    <visual>
      <origin xyz="0.125 200 0" rpy="0 0 0" />
      <geometry>
  14    <box size=".1 1 2"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

    <collision>
      <origin xyz="0.125 200 0" rpy="0 0 0" />
      <geometry>
  14    <box size=".1 1 2"/>
      </geometry>
    </collision>
  </link>

  <link name="obstacle_arms_background">
    <visual>
      <origin xyz="0 200 0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 200 0" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 200 -0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 1" />
      </material>
    </visual>

    <collision>
      <origin xyz="0 200 0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
    </collision>

    <collision>
      <origin xyz="0 200 0" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
    </collision>

    <collision>
      <origin xyz="0 200 -0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 1 0.1"/>
      </geometry>
    </collision>
  </link>
  
  <joint name="obstacle_base_glue" type="fixed">
    <origin xyz="0 0 0" rpy="0 0 0"/>
    <parent link="obstacle_base_background" />
    <child link="obstacle_arms_background" />
    <axis xyz="0 -1 0" />
  </joint>
  </robot>
</xml>
"""

In [7]:
obstacle = """
<?xml version="1.0"?><robot name="Obstacle">
  <link name="obstacle_base">
    <visual>
      <origin xyz="0.125 0 0" rpy="0 0 0" />
      <geometry>
  14    <box size=".1 100 2"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -3" rpy="0 0 0" />
      <geometry>
  14    <box size="100 100 2"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <visual>
      <origin xyz="-3 0 0" rpy="0 0 0" />
      <geometry>
  14    <box size="1 100 100"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <visual>
      <origin xyz="3 0 0" rpy="0 0 0" />
      <geometry>
  14    <box size="1 100 100"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 5 0" rpy="0 0 0" />
      <geometry>
  14    <box size="100 1 100"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <collision>
      <origin xyz="0.125 0 0" rpy="0 0 0" />
      <geometry>
  14    <box size=".1 100 2"/>
      </geometry>
    </collision>

    <collision>
      <origin xyz="0 0 -3" rpy="0 0 0" />
      <geometry>
  14    <box size="100 100 2"/>
      </geometry>
    </collision>

    <collision>
      <origin xyz="-3 0 0" rpy="0 0 0" />
      <geometry>
  14    <box size="1 100 100"/>
      </geometry>
    </collision>

    <collision>
      <origin xyz="3 0 0" rpy="0 0 0" />
      <geometry>
  14    <box size="1 100 100"/>
      </geometry>
    </collision>

    <collision>
      <origin xyz="0 5 0" rpy="0 0 0" />
      <geometry>
  14    <box size="100 1 100"/>
      </geometry>
    </collision>
  </link>

  <link name="obstacle_arms">
    <visual>
      <origin xyz="0 0 0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 0" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <visual>
      <origin xyz="0 0 -0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
      <material>
        <color rgba="0 0 1 0" />
      </material>
    </visual>

    <collision>
      <origin xyz="0 0 0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
    </collision>

    <collision>
      <origin xyz="0 0 0" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
    </collision>

    <collision>
      <origin xyz="0 0 -0.5" rpy="0 0.2 0" />
      <geometry>
  14    <box size="0.25 100 0.1"/>
      </geometry>
    </collision>
  </link>
  
  <joint name="obstacle_glue" type="fixed">
    <origin xyz="0 0 0" rpy="0 0 0"/>
    <parent link="obstacle_base" />
    <child link="obstacle_arms" />
    <axis xyz="0 -1 0" />
  </joint>

  </robot>
</xml>
"""

## Testing Stuff

In [13]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(climber, "urdf")
visualizer.parser().AddModelsFromString(obstacle_background, "urdf")
visualizer.parser().AddModelsFromString(obstacle, "urdf")
visualizer.Run(loop_once=False)
meshcat.Delete()
meshcat.DeleteAddedControls()

INFO:drake:Click 'Stop Running' or press Esc to quit


In [18]:
def HybridClimber():
    builder = RobotDiagramBuilder()
    
    # Load the climber 
    builder.parser().AddModelsFromString(climber, "urdf")

    # Load the obstacle
    builder.parser().AddModelsFromString(obstacle_background, "urdf")
    builder.parser().AddModelsFromString(obstacle, "urdf")

    plant = builder.plant()

    obstacle_frame = plant.GetFrameByName("obstacle_base")
    plant.WeldFrames(plant.world_frame(), obstacle_frame)
    obstacle_background_frame = plant.GetFrameByName("obstacle_base_background")
    plant.WeldFrames(plant.world_frame(), obstacle_background_frame)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = diagram.plant().GetMyContextFromRoot(context)

    print(plant.GetStateNames())

    kMinTimeStep = 0.05
    kMaxTimeStep = 1.0
    hybrid = HybridMultibodyCollocation(diagram, context, kMinTimeStep,
                                    kMaxTimeStep)
    prog = hybrid.prog()

    print(hybrid.GetContactPairCandidates())
    print(len(hybrid.GetContactPairCandidates()))

    kNumTimeSteps = 5
    aerial_phase = hybrid.AddMode("aerial", kNumTimeSteps, set())
    aerial_phase.AddEqualTimeIntervalsConstraints()
    contact_modes = []
    for i in range(len(hybrid.GetContactPairCandidates())):
        contact_phase = hybrid.AddModeWithInelasticImpact(
            f"contact_{i}", kNumTimeSteps, hybrid.GetContactPairCandidates().pop())
        contact_phase.AddEqualTimeIntervalsConstraints()
        contact_modes.append(contact_phase)
    
    print(len(contact_modes))
    print(aerial_phase.state())

    # ['world-hand_x', 'world-hand_y', 'world-hand_qz', 'theta0_q', 'theta1_q', 'theta2_q', 
    #  'world-hand_vx', 'world-hand_vy', 'world-hand_wz', 'theta0_w', 'theta1_w', 'theta2_w']
    # NOTE: world-hand_x is Z, world-hand_y is X
    initial_state = [0.1, 0.02, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
    prog.AddBoundingBoxConstraint(initial_state, initial_state, aerial_phase.initial_state())

    final_state = [0.6, 0.02, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
    prog.AddBoundingBoxConstraint(final_state, final_state, aerial_phase.final_state())

    u_aerial = aerial_phase.input()
    u_contact_modes = []
    for mode in contact_modes:
        u_contact_modes.append(mode.input())

    result = Solve(prog)

    print('aerial phase [time, state]')
    aerial_traj = aerial_phase.ReconstructStateTrajectory(result)
    print(np.vstack((aerial_traj.get_segment_times(), 
                     aerial_traj.vector_values(aerial_traj.get_segment_times()))).T)

    for i in range(len(contact_modes)):
        mode = contact_modes[i]
        print(f'contact_{i} phase [time, state]')
        contact_traj = mode.ReconstructStateTrajectory(result)
        print(np.vstack((contact_traj.get_segment_times(), 
                        contact_traj.vector_values(contact_traj.get_segment_times()))).T)
                        
    traj = hybrid.ReconstructStateTrajectory(result)
    force = hybrid.ReconstructContactForceTrajectory(
        result, hybrid.GetContactPairCandidates().pop())

    print('all phases [time, state, contact force]')
    print(np.vstack((traj.get_segment_times(), 
                    traj.vector_values(traj.get_segment_times()),
                    force.vector_values(traj.get_segment_times()))).T)

    if not result.is_success():
        print(result.GetInfeasibleConstraintNames(prog))

    return hybrid, result

hybrid, result = HybridClimber()

['world-hand_x', 'world-hand_y', 'world-hand_qz', 'theta0_q', 'theta1_q', 'theta2_q', 'world-hand_vx', 'world-hand_vy', 'world-hand_wz', 'theta0_w', 'theta1_w', 'theta2_w']
{(<GeometryId value=2173>, <GeometryId value=2267>), (<GeometryId value=2173>, <GeometryId value=2251>), (<GeometryId value=2173>, <GeometryId value=2270>), (<GeometryId value=2173>, <GeometryId value=2217>), (<GeometryId value=2173>, <GeometryId value=2201>), (<GeometryId value=2173>, <GeometryId value=2264>), (<GeometryId value=2173>, <GeometryId value=2214>), (<GeometryId value=2173>, <GeometryId value=2220>), (<GeometryId value=2173>, <GeometryId value=2239>), (<GeometryId value=2173>, <GeometryId value=2245>), (<GeometryId value=2173>, <GeometryId value=2242>), (<GeometryId value=2173>, <GeometryId value=2248>)}
12
12
[Variable('x(0)', Continuous) Variable('x(1)', Continuous)
 Variable('x(2)', Continuous) Variable('x(3)', Continuous)
 Variable('x(4)', Continuous) Variable('x(5)', Continuous)
 Variable('x(6)', C

In [17]:
traj = hybrid.ReconstructStateTrajectory(result)
force = hybrid.ReconstructContactForceTrajectory(
    result, hybrid.GetContactPairCandidates().pop())

print('all phases [time, state, contact force]')
print(np.vstack((traj.get_segment_times(), 
                traj.vector_values(traj.get_segment_times()),
                force.vector_values(traj.get_segment_times()))).T)

NameError: name 'hybrid' is not defined

KernelInterrupted: Execution interrupted by the Jupyter kernel.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=067a6c9a-b930-4c28-8084-e86f86716a80' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>